In [1]:
from transformers import AutoTokenizer
import transformers
import torch

model_id = "codellama/CodeLlama-7b-hf"

/home/adam-torek/miniconda3/envs/EXLLMQL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
pipeline = transformers.pipeline("text-generation", 
                                 model=model_id, 
                                 tokenizer=tokenizer, 
                                 torch_dtype=torch.float16, 
                                 device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.82s/it]


In [6]:
def generate_code(prompt):
    sequences = pipeline(prompt, 
                         do_sample=True, 
                         temperature=0.1, 
                         top_p=0.9, 
                         num_return_sequences = 1, 
                         eos_token_id = tokenizer.eos_token_id, 
                         max_length=512,
                         truncation=True
                        )
    return sequences[0]['generated_text']

In [8]:
from itertools import islice
from evalplus.data import get_human_eval_plus, write_jsonl

samples = [
    dict(task_id=task_id, solution=generate_code(problem["prompt"]))
    for task_id, problem in islice(get_human_eval_plus().items(), 0, 20, 1)
]
write_jsonl("samples.jsonl", samples)

TypeError: islice() takes no keyword arguments